In [ ]:
#@markdown <br><center><img src='https://github.com/TheCaduceus/Google-Drive-Cloner/blob/main/Img/st.jpeg?raw=true' height="50" alt="ST-logo"/></center>
#@markdown <center><h3>Check Tranfer Speed!</h3></center><br>
!curl -s https://raw.githubusercontent.com/sivel/speedtest-cli/master/speedtest.py | python

In [ ]:
#@markdown <br><center><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png" height="50" alt="Gdrive-logo' "/></center>
#@markdown <center><h3>Connect to Google Account</h3></center><br>
#Mount your Gdrive! 
from google.colab import drive
drive.mount._DEBUG = False
drive.mount('/gdrive', force_remount=True)

In [ ]:
#@markdown <br><center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3>Select zipped file in My Drive</h3></center><br>
import sys
import os
import ipywidgets as ui
from IPython.display import display

class PathSelector():
    def __init__(self,start_dir,select_file=True):
        self.file        = None 
        self.select_file = select_file
        self.cwd         = start_dir
        self.select      = ui.SelectMultiple(options=['init'],value=(),rows=10,description='') 
        self.accord      = ui.Accordion(children=[self.select]) 

        self.accord.selected_index = None # Start closed (showing path only)
        self.refresh(self.cwd)
        self.select.observe(self.on_update,'value')

    def on_update(self,change):
        if len(change['new']) > 0:
            self.refresh(change['new'][0])

    def refresh(self,item):
        path = os.path.abspath(os.path.join(self.cwd,item))

        if os.path.isfile(path):
            if self.select_file:
                self.accord.set_title(0,path)  
                self.file = path
                self.accord.selected_index = None
            else:
                self.select.value = ()

        else:
            self.file = None 
            self.cwd  = path

            # Build list of files and dirs
            keys = ['[..]']; 
            for item in os.listdir(path):
                if item[0] == '.':
                    continue
                elif os.path.isdir(os.path.join(path,item)):
                    keys.append('['+item+']'); 
                else:
                    keys.append(item); 

            keys.sort(key=str.lower)
            vals = []
            for k in keys:
                if k[0] == '[':
                    vals.append(k[1:-1])
                else:
                    vals.append(k)

            # Update widget
            self.accord.set_title(0,path)  
            self.select.options = list(zip(keys,vals)) 
            with self.select.hold_trait_notifications():
                self.select.value = ()

f = PathSelector('/gdrive/MyDrive')
display(f.accord)

In [ ]:
#@markdown <br><center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3>Run to Extract files</h3></center><br>

!pip install patool pyunpack
!apt-get install p7zip-full

import pyunpack as unzip

dirPath = os.path.splitext(f.file)[0]
os.makedirs(dirPath, exist_ok=True)

unzip.Archive(f.file).extractall(dirPath)

print("\n\n---------------------------------")
print("Extracted Files: \n")
os.listdir(dirPath)

In [ ]:
#@markdown <br><center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3>Save and Disconnect from Google Drive</h3></center><br>

try:
    drive.flush_and_unmount()
except ValueError:
    pass